In [16]:
from src.search.paragraphs import DocumentStore, Document
from src.search.passage_ranking import ResponseScorer

In [4]:
host = 'http://172.31.18.19:9200'

In [19]:
index = 'tmp_es_template' # you have to change the index name to something like '<your-firstname>-blahblah'

In [20]:
storeclient = DocumentStore(host)

In [21]:
storeclient.create_store(index, lang='en', reset_index=True)

INFO:elasticsearch:DELETE http://172.31.18.19:9200/tmp_es_template [status:200 request:0.035s]
INFO:elasticsearch:PUT http://172.31.18.19:9200/tmp_es_template [status:200 request:0.086s]


In [22]:
with open('sample.txt') as f:
    data = [Document(text=s.strip(), id=i) for i, s in enumerate(f.readlines())]

In [23]:
data

[Document(text="Napoleon Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history.[3][4]", id=0),
 Document(text="Born in Maida Vale, London, Turing was raised in southern England. He graduated at King's College, Cambridge, with a degree in mathematics. Whilst he was a fellow at Cambridge, he published a proof demonstrating that some purely mathematical yes–no questions can never be answered by computation and defined a Turing machine, and went on to prove the ha

In [24]:
storeclient.add_documents(index=index, documents=data)

INFO:elasticsearch:POST http://172.31.18.19:9200/_bulk?refresh=true [status:200 request:0.013s]


In [25]:
query = 'Napoleon'
#query = 'mediteranen'
#query = 'civilizatio'

In [26]:
print('search results:')
print(storeclient.search(index, query))
storeclient.suggest(index=index, query_string=query)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.004s]
INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.002s]


search results:
[SearchResult(paragraph=Document(text="Napoleon Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history.[3][4]", id='0'), highlight=['<b>Napoleon</b> Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the'], score=0.89166284)]


([{'text': 'napoleon', 'offset': 0, 'length': 8, 'options': []}],
 [{'text': 'napoleon', 'offset': 0, 'length': 8, 'options': []}])

In [27]:
scorer = ResponseScorer()

Some weights of the model checkpoint at models/asnq_roberta were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
storeclient.search(index, query, ranker=scorer)

INFO:elasticsearch:POST http://172.31.18.19:9200/tmp_es_template/_search [status:200 request:0.003s]
INFO:root:Score passages...
INFO:root:Prepare data: 0.00s
100%|██████████| 1/1 [00:00<00:00, 62.46it/s]
INFO:root:Inference: 0.06


[SearchResult(paragraph=Document(text="Napoleon Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the French Revolution and led several successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804. As Napoleon I, he was Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy has endured, and he has been one of the most celebrated and controversial leaders in world history.[3][4]", id='0'), highlight=['<b>Napoleon</b> Bonaparte[a] (born Napoleone Buonaparte; 15 August 1769 – 5 May 1821), and later known by his regnal name Napoleon I, was a French military and political leader who rose to prominence during the'], score=0.4489596486091614)]

In [ ]:
[ doc1, doc2, ..., docn ] -> elasticsearch's index -> query
es:
- use tf-idf (bm25) -> [ doc_i1, ..., doc_ij ]
- use QA model: (query, doc) -> 0 <= score <= 1
    [(doc_i1, sc1), ..., (doc_ij, scj)]